In [1]:
import pandas as pd
import numpy as np
import tqdm
from tqdm import tqdm
import glob
import os
import matplotlib.pyplot as plt
import warnings

# 忽略特定类型的警告

warnings.filterwarnings('ignore', category=FutureWarning)


In [ ]:

folder_path = 'AshareData'
# 初始化一个空的 DataFrame
price_df = pd.DataFrame()
# 遍历指定路径下的所有文件
for file_name in tqdm(os.listdir(folder_path)):
    if file_name.endswith('.feather'):  # 修改为你需要的文件类型
        file_path = os.path.join(folder_path, file_name)
        # 读取文件并追加到 DataFrame 中
        df = pd.read_feather(file_path)
        price_df = pd.concat([price_df, df], ignore_index=True)

# price_df.to_feather("ASharePriceDf.feather")

In [ ]:

# folder_path = 'FundamentalData'
# # 初始化一个空的 DataFrame
# price_df = pd.DataFrame()
# # 遍历指定路径下的所有文件
# for file_name in tqdm(os.listdir(folder_path)):
#     if file_name.endswith('.pkl'):  # 修改为你需要的文件类型
#         file_path = os.path.join(folder_path, file_name)
#         # 读取文件并追加到 DataFrame 中
#         df = pd.read_pickle(file_path)
#         price_df = pd.concat([price_df, df], ignore_index=True)
# save_path = 'fundamental_data_by_tiems'
# price_df['item_title_clean'] = price_df['item_title'].str.replace(r'[\/\(\):]', '_', regex=True)

# # 遍历分组
# for item_field, group_data in tqdm(price_df.groupby('item_title_clean')):
#     file_path = os.path.join(save_path, f"{item_field}.pkl")
#     group_data.to_pickle(file_path)

In [2]:
price_df = pd.read_feather("ASharePriceDf.feather")
price_df.rename(columns={'trade_date': 'Date'}, inplace=True)
price_df.rename(columns={'ts_code': 'stock_id'}, inplace=True)
price_df.rename(columns={'open': 'DS_OPEN'}, inplace=True)
price_df.rename(columns={'high': 'DS_HIGH'}, inplace=True)
price_df.rename(columns={'low': 'DS_LOW'}, inplace=True)
price_df.rename(columns={'close': 'DS_CLOSE'}, inplace=True)
price_df.rename(columns={'vol': 'DS_VOLUME'}, inplace=True)
price_df = price_df.reindex(columns=['Date','stock_id', 'DS_OPEN', 'DS_HIGH', 'DS_LOW', 'DS_CLOSE',
       'pre_close', 'change', 'pct_chg', 'DS_VOLUME', 'amount'])


In [3]:
price_df

,Date,stock_id,DS_OPEN,DS_HIGH,DS_LOW,DS_CLOSE,pre_close,change,pct_chg,DS_VOLUME,amount
0,20100104,000798.SZ,8.11,8.36,8.10,8.15,8.01,0.14,1.7500,94909.87,77965.8597
1,20100104,600405.SH,9.90,10.00,9.73,9.90,9.78,0.12,1.2300,73226.11,72576.1880
2,20100104,600590.SH,15.53,16.27,15.53,15.87,15.43,0.44,2.8500,115549.64,184679.2390
3,20100104,600694.SH,44.40,44.42,43.00,43.77,43.79,-0.02,-0.0500,27084.63,118864.4670
4,20100104,600207.SH,5.83,5.87,5.71,5.72,5.83,-0.11,-1.8900,48736.57,28113.0040
...,...,...,...,...,...,...,...,...,...,...,...
11766812,20240923,603341.SH,36.25,36.88,35.45,35.47,35.82,-0.35,-0.9771,33223.55,119485.9840
11766813,20240923,000677.SZ,2.52,2.55,2.48,2.52,2.54,-0.02,-0.7874,144766.00,36352.0160
11766814,20240923,300415.SZ,18.08,18.45,17.91,18.11,18.05,0.06,0.3324,39012.00,71091.5630
11766815,20240923,002360.SZ,4.18,4.22,4.14,4.19,4.18,0.01,0.2392,12726.00,5317.1780


In [ ]:
price_df = price_df.drop(['pre_close','change','pct_chg','amount'], axis=1)
price_df

In [ ]:
# # step1: 删掉为空的数值
# price_df = price_df.dropna(subset=['DS_OPEN'])
# price_df = price_df.dropna(subset=['DS_HIGH'])
# price_df = price_df.dropna(subset=['DS_LOW'])
# price_df = price_df.dropna(subset=['DS_CLOSE'])
# price_df.rename(columns={'DS_OPEN': 'AdjOpen'}, inplace=True)
# price_df.rename(columns={'DS_HIGH': 'AdjHigh'}, inplace=True)
# price_df.rename(columns={'DS_LOW': 'AdjLow'}, inplace=True)
# price_df.rename(columns={'DS_CLOSE': 'AdjClose'}, inplace=True)
# price_df = price_df.drop(['DS_TOTRET','DS_TICKER','DS_FIRST_PRICE_DATE','DS_MKTCAP','DS_VWAP'], axis=1)
# price_df

In [ ]:
# 剔除数据值不足60个的股票
price_df = price_df.groupby('stock_id').filter(lambda x: len(x) >= 60)
price_df

In [ ]:
# step3: 排序
price_df['Date'] = pd.to_datetime(price_df['Date'])
price_df = price_df.sort_values(by=['stock_id', 'Date'], ascending=True)
price_df

In [ ]:
# 以下开始计算因子
price_df['Avg50'] = price_df['AdjClose'] / price_df.groupby('stock_id')['AdjClose'].shift(50) - 1
price_df

In [ ]:
price_df['Avg100'] = price_df['AdjClose'] / price_df.groupby('stock_id')['AdjClose'].shift(100) - 1
price_df['Avg200'] = price_df['AdjClose'] / price_df.groupby('stock_id')['AdjClose'].shift(200) - 1
price_df

In [ ]:
import ta
import numpy as np

def calculate_atr(group):
    if len(group) >= 14:
        atr = ta.volatility.AverageTrueRange(high=group['AdjHigh'], low=group['AdjLow'], close=group['AdjClose'], window=14)
        return atr.average_true_range()
    else:
        return pd.Series([np.nan] * len(group), index=group.index)
tqdm.pandas()

price_df['volatility_atr'] = price_df.groupby('stock_id').progress_apply(calculate_atr).reset_index(level=0, drop=True)

price_df

In [ ]:
from ta.volatility import BollingerBands


def calculate_bbm(group):
    if len(group) >= 20:
        indicator_bb = BollingerBands(close=group["AdjClose"], window=20, window_dev=2)
        return indicator_bb.bollinger_mavg()
    else:
        return pd.Series([np.nan] * len(group), index=group.index)
price_df['volatility_bbm'] = price_df.groupby('stock_id').progress_apply(calculate_bbm).reset_index(level=0, drop=True)


def calculate_bbh(group):
    if len(group) >= 20:
        indicator_bb = BollingerBands(close=group["AdjClose"], window=20, window_dev=2)
        return indicator_bb.bollinger_hband()
    else:
        return pd.Series([np.nan] * len(group), index=group.index)

price_df['volatility_bbh'] = price_df.groupby('stock_id').progress_apply(calculate_bbh).reset_index(level=0, drop=True)


def calculate_bbl(group):
    if len(group) >= 20:
        indicator_bb = BollingerBands(close=group["AdjClose"], window=20, window_dev=2)
        return indicator_bb.bollinger_lband()
    else:
        return pd.Series([np.nan] * len(group), index=group.index)

price_df['volatility_bbl'] = price_df.groupby('stock_id').progress_apply(calculate_bbl).reset_index(level=0, drop=True)


def calculate_bbhi(group):
    if len(group) >= 20:
        indicator_bb = BollingerBands(close=group["AdjClose"], window=20, window_dev=2)
        return indicator_bb.bollinger_hband_indicator()
    else:
        return pd.Series([np.nan] * len(group), index=group.index)

price_df['volatility_bbhi'] = price_df.groupby('stock_id').progress_apply(calculate_bbhi).reset_index(level=0, drop=True)


def calculate_bbli(group):
    if len(group) >= 20:
        indicator_bb = BollingerBands(close=group["AdjClose"], window=20, window_dev=2)
        return indicator_bb.bollinger_lband_indicator()
    else:
        return pd.Series([np.nan] * len(group), index=group.index)

price_df['volatility_bbli'] = price_df.groupby('stock_id').progress_apply(calculate_bbli).reset_index(level=0, drop=True)


def calculate_bbp(group):
    if len(group) >= 20:
        indicator_bb = BollingerBands(close=group["AdjClose"], window=20, window_dev=2)
        return indicator_bb.bollinger_pband()
    else:
        return pd.Series([np.nan] * len(group), index=group.index)

price_df['volatility_bbp'] = price_df.groupby('stock_id').progress_apply(calculate_bbp).reset_index(level=0, drop=True)


def calculate_bbw(group):
    if len(group) >= 20:
        indicator_bb = BollingerBands(close=group["AdjClose"], window=20, window_dev=2)
        return indicator_bb.bollinger_wband()
    else:
        return pd.Series([np.nan] * len(group), index=group.index)

price_df['volatility_bbw'] = price_df.groupby('stock_id').progress_apply(calculate_bbw).reset_index(level=0, drop=True)


price_df

In [ ]:
from ta.volatility import KeltnerChannel


def calculate_kch(group):
    if len(group) >= 20:
        indicator_kc = KeltnerChannel(high=group['AdjHigh'], low=group['AdjLow'], close=group['AdjClose'], window=20, window_atr=10)
        return indicator_kc.keltner_channel_hband()
    else:
        return pd.Series([np.nan] * len(group), index=group.index)

price_df['volatility_kch'] = price_df.groupby('stock_id').progress_apply(calculate_kch).reset_index(level=0, drop=True)


def calculate_kcl(group):
    if len(group) >= 20:
        indicator_kc = KeltnerChannel(high=group['AdjHigh'], low=group['AdjLow'], close=group['AdjClose'], window=20, window_atr=10)
        return indicator_kc.keltner_channel_lband()
    else:
        return pd.Series([np.nan] * len(group), index=group.index)

price_df['volatility_kcl'] = price_df.groupby('stock_id').progress_apply(calculate_kcl).reset_index(level=0, drop=True)


def calculate_kcw(group):
    if len(group) >= 20:
        indicator_kc = KeltnerChannel(high=group['AdjHigh'], low=group['AdjLow'], close=group['AdjClose'], window=20, window_atr=10)
        return indicator_kc.keltner_channel_wband()
    else:
        return pd.Series([np.nan] * len(group), index=group.index)

price_df['volatility_kcw'] = price_df.groupby('stock_id').progress_apply(calculate_kcw).reset_index(level=0, drop=True)


def calculate_kcp(group):
    if len(group) >= 20:
        indicator_kc = KeltnerChannel(high=group['AdjHigh'], low=group['AdjLow'], close=group['AdjClose'], window=20, window_atr=10)
        return indicator_kc.keltner_channel_pband()
    else:
        return pd.Series([np.nan] * len(group), index=group.index)

price_df['volatility_kcp'] = price_df.groupby('stock_id').progress_apply(calculate_kcp).reset_index(level=0, drop=True)


def calculate_kcli(group):
    if len(group) >= 20:
        indicator_kc = KeltnerChannel(high=group['AdjHigh'], low=group['AdjLow'], close=group['AdjClose'], window=20, window_atr=10)
        return indicator_kc.keltner_channel_lband_indicator()
    else:
        return pd.Series([np.nan] * len(group), index=group.index)

price_df['volatility_kcli'] = price_df.groupby('stock_id').progress_apply(calculate_kcli).reset_index(level=0, drop=True)


def calculate_kchi(group):
    if len(group) >= 20:
        indicator_kc = KeltnerChannel(high=group['AdjHigh'], low=group['AdjLow'], close=group['AdjClose'], window=20, window_atr=10)
        return indicator_kc.keltner_channel_hband_indicator()
    else:
        return pd.Series([np.nan] * len(group), index=group.index)

price_df['volatility_kchi'] = price_df.groupby('stock_id').progress_apply(calculate_kchi).reset_index(level=0, drop=True)


def calculate_kcm(group):
    if len(group) >= 20:
        indicator_kc = KeltnerChannel(high=group['AdjHigh'], low=group['AdjLow'], close=group['AdjClose'], window=20, window_atr=10)
        return indicator_kc.keltner_channel_mband()
    else:
        return pd.Series([np.nan] * len(group), index=group.index)

price_df['volatility_kcm'] = price_df.groupby('stock_id').progress_apply(calculate_kcm).reset_index(level=0, drop=True)


price_df

In [ ]:
from ta.volatility import DonchianChannel


def calculate_dch(group):
    if len(group) >= 20:
        indicator_dc = DonchianChannel(high=group['AdjHigh'], low=group['AdjLow'], close=group['AdjClose'], window=20)
        return indicator_dc.donchian_channel_hband()
    else:
        return pd.Series([np.nan] * len(group), index=group.index)

price_df['volatility_dch'] = price_df.groupby('stock_id').progress_apply(calculate_dch).reset_index(level=0, drop=True)


def calculate_dcl(group):
    if len(group) >= 20:
        indicator_dc = DonchianChannel(high=group['AdjHigh'], low=group['AdjLow'], close=group['AdjClose'], window=20)
        return indicator_dc.donchian_channel_lband()
    else:
        return pd.Series([np.nan] * len(group), index=group.index)

price_df['volatility_dcl'] = price_df.groupby('stock_id').progress_apply(calculate_dcl).reset_index(level=0, drop=True)


def calculate_dcm(group):
    if len(group) >= 20:
        indicator_dc = DonchianChannel(high=group['AdjHigh'], low=group['AdjLow'], close=group['AdjClose'], window=20)
        return indicator_dc.donchian_channel_mband()
    else:
        return pd.Series([np.nan] * len(group), index=group.index)

price_df['volatility_dcm'] = price_df.groupby('stock_id').progress_apply(calculate_dcm).reset_index(level=0, drop=True)


def calculate_dcp(group):
    if len(group) >= 20:
        indicator_dc = DonchianChannel(high=group['AdjHigh'], low=group['AdjLow'], close=group['AdjClose'], window=20)
        return indicator_dc.donchian_channel_pband()
    else:
        return pd.Series([np.nan] * len(group), index=group.index)

price_df['volatility_dcp'] = price_df.groupby('stock_id').progress_apply(calculate_dcp).reset_index(level=0, drop=True)


def calculate_dcw(group):
    if len(group) >= 20:
        indicator_dc = DonchianChannel(high=group['AdjHigh'], low=group['AdjLow'], close=group['AdjClose'], window=20)
        return indicator_dc.donchian_channel_wband()
    else:
        return pd.Series([np.nan] * len(group), index=group.index)

price_df['volatility_dcw'] = price_df.groupby('stock_id').progress_apply(calculate_dcw).reset_index(level=0, drop=True)


price_df

In [ ]:
from ta.volatility import UlcerIndex


def calculate_ui(group):
    if len(group) >= 14:
        indicator_ui = UlcerIndex(close=group['AdjClose'], window=14)
        return indicator_ui.ulcer_index() 
    else:
        return pd.Series([np.nan] * len(group), index=group.index)

price_df['volatility_ui'] = price_df.groupby('stock_id').progress_apply(calculate_ui).reset_index(level=0, drop=True)


price_df

In [ ]:
from ta.trend import ADXIndicator


def calculate_adx(group):
    if len(group) >= 14:
        indicator_adx = ADXIndicator(high=group['AdjHigh'], low=group['AdjLow'], close=group['AdjClose'], window=14)
        return indicator_adx.adx()
    else:
        return pd.Series([np.nan] * len(group), index=group.index)

price_df['trend_adx'] = price_df.groupby('stock_id').progress_apply(calculate_adx).reset_index(level=0, drop=True)


def calculate_adx_neg(group):
    if len(group) >= 14:
        indicator_adx = ADXIndicator(high=group['AdjHigh'], low=group['AdjLow'], close=group['AdjClose'], window=14)
        return indicator_adx.adx_neg()
    else:
        return pd.Series([np.nan] * len(group), index=group.index)

price_df['trend_adx_neg'] = price_df.groupby('stock_id').progress_apply(calculate_adx_neg).reset_index(level=0, drop=True)


def calculate_adx_pos(group):
    if len(group) >= 14:
        indicator_adx = ADXIndicator(high=group['AdjHigh'], low=group['AdjLow'], close=group['AdjClose'], window=14)
        return indicator_adx.adx_pos()
    else:
        return pd.Series([np.nan] * len(group), index=group.index)

price_df['trend_adx_pos'] = price_df.groupby('stock_id').progress_apply(calculate_adx_pos).reset_index(level=0, drop=True)


price_df

In [ ]:
from ta.volume import AccDistIndexIndicator

def calculate_adi(group):
    indicator_adi = AccDistIndexIndicator(high=group['AdjHigh'], low=group['AdjLow'], close=group['AdjClose'], volume=group['DS_VOLUME'])
    return indicator_adi.acc_dist_index()

price_df['volume_adi'] = price_df.groupby('stock_id').progress_apply(calculate_adi).reset_index(level=0, drop=True)
price_df

In [ ]:
from ta.volume import OnBalanceVolumeIndicator

def calculate_obv(group):
    indicator_obv = OnBalanceVolumeIndicator(close=group['AdjClose'], volume=group['DS_VOLUME'])
    return indicator_obv.on_balance_volume()

price_df['volume_obv'] = price_df.groupby('stock_id').progress_apply(calculate_obv).reset_index(level=0, drop=True)
price_df

In [ ]:
from ta.volume import ChaikinMoneyFlowIndicator

def calculate_cmf(group):
    indicator_cmf = ChaikinMoneyFlowIndicator(high=group['AdjHigh'], low=group['AdjLow'], close=group['AdjClose'], volume=group['DS_VOLUME'], window=20)
    return indicator_cmf.chaikin_money_flow()

price_df['volume_cmf'] = price_df.groupby('stock_id').progress_apply(calculate_cmf).reset_index(level=0, drop=True)
price_df

In [ ]:
from ta.volume import ForceIndexIndicator

def calculate_fi(group):
    indicator_fi = ForceIndexIndicator(close=group['AdjClose'], volume=group['DS_VOLUME'], window=13)
    return indicator_fi.force_index()

price_df['volume_fi'] = price_df.groupby('stock_id').progress_apply(calculate_fi).reset_index(level=0, drop=True)
price_df

In [ ]:
from ta.volume import MFIIndicator

def calculate_mfi(group):
    indicator_mfi = MFIIndicator(high=group['AdjHigh'], low=group['AdjLow'], close=group['AdjClose'], volume=group['DS_VOLUME'], window=14)
    return indicator_mfi.money_flow_index()

price_df['volume_mfi'] = price_df.groupby('stock_id').progress_apply(calculate_mfi).reset_index(level=0, drop=True)
price_df

In [ ]:
from ta.volume import EaseOfMovementIndicator

def calculate_EoM(group):
    indicator_em = EaseOfMovementIndicator(high=group['AdjHigh'], low=group['AdjLow'], volume=group['DS_VOLUME'], window=14)
    return indicator_em.ease_of_movement()

price_df['volume_em'] = price_df.groupby('stock_id').progress_apply(calculate_EoM).reset_index(level=0, drop=True)


def calculate_EMV(group):
    indicator_em = EaseOfMovementIndicator(high=group['AdjHigh'], low=group['AdjLow'], volume=group['DS_VOLUME'], window=14)
    return indicator_em.sma_ease_of_movement()

price_df['volume_sma_em'] = price_df.groupby('stock_id').progress_apply(calculate_EMV).reset_index(level=0, drop=True)


price_df

In [ ]:
from ta.volume import NegativeVolumeIndexIndicator

def calculate_nvi(group):
    indicator_nvi = NegativeVolumeIndexIndicator(close=group['AdjClose'], volume=group['DS_VOLUME'])
    return indicator_nvi.negative_volume_index()

price_df['volume_nvi'] = price_df.groupby('stock_id').progress_apply(calculate_nvi).reset_index(level=0, drop=True)
price_df

In [ ]:
from ta.volume import VolumePriceTrendIndicator

def calculate_vpt(group):
    indicator_vpt = VolumePriceTrendIndicator(close=group['AdjClose'], volume=group['DS_VOLUME'])
    return indicator_vpt.volume_price_trend()

price_df['volume_vpt'] = price_df.groupby('stock_id').progress_apply(calculate_vpt).reset_index(level=0, drop=True)
price_df

In [ ]:
from ta.volume import VolumeWeightedAveragePrice

def calculate_vwap(group):
    indicator_vwap = VolumeWeightedAveragePrice(high=group['AdjHigh'], low=group['AdjLow'], close=group['AdjClose'], volume=group['DS_VOLUME'], window=14)
    return indicator_vwap.volume_weighted_average_price()

price_df['volume_vwap'] = price_df.groupby('stock_id').progress_apply(calculate_vwap).reset_index(level=0, drop=True)
price_df

In [ ]:
from ta.trend import AroonIndicator

def calculate_aroon_ind(group):
    indicator_aroon = AroonIndicator(high=group['AdjHigh'], low=group['AdjLow'], window=25)
    return indicator_aroon.aroon_indicator()

price_df['trend_aroon_ind'] = price_df.groupby('stock_id').progress_apply(calculate_aroon_ind).reset_index(level=0, drop=True)


def calculate_aroon_down(group):
    indicator_aroon = AroonIndicator(high=group['AdjHigh'], low=group['AdjLow'], window=25)
    return indicator_aroon.aroon_down()

price_df['trend_aroon_down'] = price_df.groupby('stock_id').progress_apply(calculate_aroon_down).reset_index(level=0, drop=True)


def calculate_aroon_up(group):
    indicator_aroon = AroonIndicator(high=group['AdjHigh'], low=group['AdjLow'], window=25)
    return indicator_aroon.aroon_up()

price_df['trend_aroon_up'] = price_df.groupby('stock_id').progress_apply(calculate_aroon_up).reset_index(level=0, drop=True)


price_df

In [ ]:
from ta.trend import CCIIndicator

def calculate_cci(group):
    indicator_cci = CCIIndicator(high=group['AdjHigh'], low=group['AdjLow'], close=group['AdjClose'], window=20)
    return indicator_cci.cci()

price_df['trend_cci'] = price_df.groupby('stock_id').progress_apply(calculate_cci).reset_index(level=0, drop=True)

price_df

In [ ]:
from ta.trend import DPOIndicator

def calculate_dpo(group):
    indicator_dpo = DPOIndicator(close=group['AdjClose'], window=20)
    return indicator_dpo.dpo()

price_df['trend_dpo'] = price_df.groupby('stock_id').progress_apply(calculate_dpo).reset_index(level=0, drop=True)

price_df

In [ ]:
from ta.trend import EMAIndicator


def calculate_ema_fast(group):
    indicator_ema_fast = EMAIndicator(close=group['AdjClose'], window=14)
    return indicator_ema_fast.ema_indicator()

price_df['trend_ema_fast'] = price_df.groupby('stock_id').progress_apply(calculate_ema_fast).reset_index(level=0, drop=True)


def calculate_ema_slow(group):
    indicator_ema_slow = EMAIndicator(close=group['AdjClose'], window=28)
    return indicator_ema_slow.ema_indicator()

price_df['trend_ema_slow'] = price_df.groupby('stock_id').progress_apply(calculate_ema_slow).reset_index(level=0, drop=True)

price_df

In [ ]:
from ta.trend import SMAIndicator

def calculate_sma_fast(group):
    indicator_sma_fast = SMAIndicator(close=group['AdjClose'], window=14)
    return indicator_sma_fast.sma_indicator()

price_df['trend_sma_fast'] = price_df.groupby('stock_id').progress_apply(calculate_sma_fast).reset_index(level=0, drop=True)


def calculate_sma_slow(group):
    indicator_sma_slow = SMAIndicator(close=group['AdjClose'], window=28)
    return indicator_sma_slow.sma_indicator()

price_df['trend_sma_slow'] = price_df.groupby('stock_id').progress_apply(calculate_sma_slow).reset_index(level=0, drop=True)


price_df

In [ ]:
from ta.trend import MACD

def calculate_macd(group):
    indicator_MACD = MACD(close=group['AdjClose'])
    return indicator_MACD.macd()

price_df['trend_macd'] = price_df.groupby('stock_id').progress_apply(calculate_macd).reset_index(level=0, drop=True)


def calculate_macd_diff(group):
    indicator_MACD = MACD(close=group['AdjClose'])
    return indicator_MACD.macd_diff()

price_df['trend_macd_diff'] = price_df.groupby('stock_id').progress_apply(calculate_macd_diff).reset_index(level=0, drop=True)


def calculate_macd_signal(group):
    indicator_MACD = MACD(close=group['AdjClose'])
    return indicator_MACD.macd_signal()

price_df['trend_macd_signal'] = price_df.groupby('stock_id').progress_apply(calculate_macd_signal).reset_index(level=0, drop=True)


price_df

In [ ]:
from ta.trend import MassIndex

def calculate_mass_index(group):
    indicator_mass_index = MassIndex(high=group['AdjHigh'], low=group['AdjLow'])
    return indicator_mass_index.mass_index()

price_df['trend_mass_index'] = price_df.groupby('stock_id').progress_apply(calculate_mass_index).reset_index(level=0, drop=True)

price_df

In [ ]:
price_df = price_df.reset_index(drop=True)
price_df

In [ ]:
from ta.trend import PSARIndicator

def calculate_psar(group):
    indicator_psar = PSARIndicator(high=group['AdjHigh'], low=group['AdjLow'], close=group['AdjClose'])
    group['trend_psar'] = indicator_psar.psar()
    group['trend_psar_down'] = indicator_psar.psar_down()
    group['trend_psar_down_indicator'] = indicator_psar.psar_down_indicator()
    group['trend_psar_up'] = indicator_psar.psar_up()
    group['trend_psar_up_indicator'] = indicator_psar.psar_up_indicator()
    return group

price_df = price_df.groupby('stock_id').progress_apply(calculate_psar).reset_index(level=0, drop=True)

price_df

In [66]:
# # 上面版本代替报错版本

# from ta.trend import PSARIndicator

# def calculate_psar(group):
#     indicator_psar = PSARIndicator(high=group['AdjHigh'], low=group['AdjLow'], close=group['AdjClose'])
#     return indicator_psar.psar()

# price_df['trend_psar'] = price_df.groupby('stock_id').progress_apply(calculate_psar).reset_index(level=0, drop=True)


# def calculate_psar_down(group):
#     indicator_psar = PSARIndicator(high=group['AdjHigh'], low=group['AdjLow'], close=group['AdjClose'])
#     return indicator_psar.psar_down()

# price_df['trend_psar_down'] = price_df.groupby('stock_id').progress_apply(calculate_psar_down).reset_index(level=0, drop=True)


# def calculate_psar_down_indicator(group):
#     indicator_psar = PSARIndicator(high=group['AdjHigh'], low=group['AdjLow'], close=group['AdjClose'])
#     return indicator_psar.psar_down_indicator()

# price_df['trend_psar_down_indicator'] = price_df.groupby('stock_id').progress_apply(calculate_psar_down_indicator).reset_index(level=0, drop=True)


# def calculate_psar_up(group):
#     indicator_psar = PSARIndicator(high=group['AdjHigh'], low=group['AdjLow'], close=group['AdjClose'])
#     return indicator_psar.psar_up()

# price_df['trend_psar_up'] = price_df.groupby('stock_id').progress_apply(calculate_psar_up).reset_index(level=0, drop=True)


# def calculate_psar_up_indicator(group):
#     indicator_psar = PSARIndicator(high=group['AdjHigh'], low=group['AdjLow'], close=group['AdjClose'])
#     return indicator_psar.psar_up_indicator()

# price_df['trend_psar_up_indicator'] = price_df.groupby('stock_id').progress_apply(calculate_psar_up_indicator).reset_index(level=0, drop=True)


# price_df

In [ ]:
from ta.trend import STCIndicator

def calculate_stc(group):
    indicator_stc = STCIndicator(close=group['AdjClose'])
    return indicator_stc.stc()

price_df['trend_stc'] = price_df.groupby('stock_id').progress_apply(calculate_stc).reset_index(level=0, drop=True)

price_df

In [ ]:
from ta.trend import WMAIndicator

def calculate_wma(group):
    indicator_wma = WMAIndicator(close=group['AdjClose'])
    return indicator_wma.wma()
price_df['trend_wma'] = price_df.groupby('stock_id').progress_apply(calculate_wma).reset_index(level=0, drop=True)

price_df

In [ ]:
from ta.trend import VortexIndicator

def calculate_vi_pos(group):
    indicator_vi = VortexIndicator(high=group['AdjHigh'], low=group['AdjLow'], close=group['AdjClose'])
    return indicator_vi.vortex_indicator_pos()

price_df['trend_vortex_ind_pos'] = price_df.groupby('stock_id').progress_apply(calculate_vi_pos).reset_index(level=0, drop=True)

def calculate_vi_neg(group):
    indicator_vi = VortexIndicator(high=group['AdjHigh'], low=group['AdjLow'], close=group['AdjClose'])
    return indicator_vi.vortex_indicator_neg()

price_df['trend_vortex_ind_neg'] = price_df.groupby('stock_id').progress_apply(calculate_vi_neg).reset_index(level=0, drop=True)


def calculate_vi_diff(group):
    indicator_vi = VortexIndicator(high=group['AdjHigh'], low=group['AdjLow'], close=group['AdjClose'])
    return indicator_vi.vortex_indicator_diff()

price_df['trend_vortex_ind_diff'] = price_df.groupby('stock_id').progress_apply(calculate_vi_diff).reset_index(level=0, drop=True)


price_df

In [ ]:
from ta.trend import TRIXIndicator

def calculate_trix(group):
    indicator_trix = TRIXIndicator(close=group['AdjClose'])
    return indicator_trix.trix()

price_df['trend_trix'] = price_df.groupby('stock_id').progress_apply(calculate_trix).reset_index(level=0, drop=True)

price_df

In [ ]:
from ta.trend import KSTIndicator

def calculate_kst(group):
    indicator_kst = KSTIndicator(close=group['AdjClose'])
    return indicator_kst.kst()

price_df['trend_kst'] = price_df.groupby('stock_id').progress_apply(calculate_kst).reset_index(level=0, drop=True)


def calculate_kst_diff(group):
    indicator_kst = KSTIndicator(close=group['AdjClose'])
    return indicator_kst.kst_diff()

price_df['trend_kst_diff'] = price_df.groupby('stock_id').progress_apply(calculate_kst_diff).reset_index(level=0, drop=True)


def calculate_kst_sig(group):
    indicator_kst = KSTIndicator(close=group['AdjClose'])
    return indicator_kst.kst_sig()

price_df['trend_kst_sig'] = price_df.groupby('stock_id').progress_apply(calculate_kst_sig).reset_index(level=0, drop=True)


price_df

In [ ]:
from ta.trend import IchimokuIndicator

def calculate_ichimoku_a(group):
    indicator_Ichimoku = IchimokuIndicator(high=group['AdjHigh'], low=group['AdjLow'])
    return indicator_Ichimoku.ichimoku_a()

price_df['trend_ichimoku_a'] = price_df.groupby('stock_id').progress_apply(calculate_ichimoku_a).reset_index(level=0, drop=True)


def calculate_ichimoku_b(group):
    indicator_Ichimoku = IchimokuIndicator(high=group['AdjHigh'], low=group['AdjLow'])
    return indicator_Ichimoku.ichimoku_b()

price_df['trend_ichimoku_b'] = price_df.groupby('stock_id').progress_apply(calculate_ichimoku_b).reset_index(level=0, drop=True)


def calculate_visual_ichimoku_a(group):
    indicator_Ichimoku = IchimokuIndicator(high=group['AdjHigh'], low=group['AdjLow'], visual=True)
    return indicator_Ichimoku.ichimoku_a()

price_df['trend_visual_ichimoku_a'] = price_df.groupby('stock_id').progress_apply(calculate_visual_ichimoku_a).reset_index(level=0, drop=True)


def calculate_visual_ichimoku_b(group):
    indicator_Ichimoku = IchimokuIndicator(high=group['AdjHigh'], low=group['AdjLow'], visual=True)
    return indicator_Ichimoku.ichimoku_b()

price_df['trend_visual_ichimoku_b'] = price_df.groupby('stock_id').progress_apply(calculate_visual_ichimoku_b).reset_index(level=0, drop=True)


def calculate_ichimoku_base(group):
    indicator_Ichimoku = IchimokuIndicator(high=group['AdjHigh'], low=group['AdjLow'])
    return indicator_Ichimoku.ichimoku_base_line()

price_df['trend_ichimoku_base'] = price_df.groupby('stock_id').progress_apply(calculate_ichimoku_base).reset_index(level=0, drop=True)


def calculate_ichimoku_conv(group):
    indicator_Ichimoku = IchimokuIndicator(high=group['AdjHigh'], low=group['AdjLow'])
    return indicator_Ichimoku.ichimoku_conversion_line()

price_df['trend_ichimoku_conv'] = price_df.groupby('stock_id').progress_apply(calculate_ichimoku_conv).reset_index(level=0, drop=True)


price_df

In [ ]:
from ta.momentum import AwesomeOscillatorIndicator

def calculate_ao(group):
    indicator_ao = AwesomeOscillatorIndicator(high=group['AdjHigh'], low=group['AdjLow'])
    return indicator_ao.awesome_oscillator()

price_df['momentum_ao'] = price_df.groupby('stock_id').progress_apply(calculate_ao).reset_index(level=0, drop=True)

price_df

In [ ]:
from ta.momentum import KAMAIndicator

def calculate_kama(group):
    indicator_kama = KAMAIndicator(close=group['AdjClose'])
    return indicator_kama.kama()

price_df['momentum_kama'] = price_df.groupby('stock_id').progress_apply(calculate_kama).reset_index(level=0, drop=True)

price_df

In [ ]:
from ta.momentum import RSIIndicator

def calculate_rsi(group):
    indicator_rsi = RSIIndicator(close=group['AdjClose'])
    return indicator_rsi.rsi()

price_df['momentum_rsi'] = price_df.groupby('stock_id').progress_apply(calculate_rsi).reset_index(level=0, drop=True)

price_df

In [ ]:
from ta.momentum import StochRSIIndicator

def calculate_stoch_rsi(group):
    indicator_stoch_rsi = StochRSIIndicator(close=group['AdjClose'])
    return indicator_stoch_rsi.stochrsi()

price_df['momentum_stoch_rsi'] = price_df.groupby('stock_id').progress_apply(calculate_stoch_rsi).reset_index(level=0, drop=True)

def calculate_stoch_rsi_d(group):
    indicator_stoch_rsi = StochRSIIndicator(close=group['AdjClose'])
    return indicator_stoch_rsi.stochrsi_d()

price_df['momentum_stoch_rsi_d'] = price_df.groupby('stock_id').progress_apply(calculate_stoch_rsi_d).reset_index(level=0, drop=True)

def calculate_stoch_rsi_k(group):
    indicator_stoch_rsi = StochRSIIndicator(close=group['AdjClose'])
    return indicator_stoch_rsi.stochrsi_k()

price_df['momentum_stoch_rsi_k'] = price_df.groupby('stock_id').progress_apply(calculate_stoch_rsi_k).reset_index(level=0, drop=True)

price_df

In [ ]:
from ta.momentum import TSIIndicator

def calculate_tsi(group):
    indicator_tsi = TSIIndicator(close=group['AdjClose'])
    return indicator_tsi.tsi()

price_df['momentum_tsi'] = price_df.groupby('stock_id').progress_apply(calculate_tsi).reset_index(level=0, drop=True)

price_df

In [ ]:
from ta.momentum import StochasticOscillator

def calculate_stoch(group):
    indicator_stoch = StochasticOscillator(high=group['AdjHigh'], low=group['AdjLow'], close=group['AdjClose'])
    return indicator_stoch.stoch()

price_df['momentum_stoch'] = price_df.groupby('stock_id').progress_apply(calculate_stoch).reset_index(level=0, drop=True)


def calculate_stoch_signal(group):
    indicator_stoch = StochasticOscillator(high=group['AdjHigh'], low=group['AdjLow'], close=group['AdjClose'])
    return indicator_stoch.stoch_signal()

price_df['momentum_stoch_signal'] = price_df.groupby('stock_id').progress_apply(calculate_stoch_signal).reset_index(level=0, drop=True)

price_df

In [ ]:
from ta.momentum import UltimateOscillator

def calculate_uo(group):
    indicator_uo = UltimateOscillator(high=group['AdjHigh'], low=group['AdjLow'], close=group['AdjClose'])
    return indicator_uo.ultimate_oscillator()

price_df['momentum_uo'] = price_df.groupby('stock_id').progress_apply(calculate_uo).reset_index(level=0, drop=True)

price_df

In [ ]:
from ta.momentum import WilliamsRIndicator

def calculate_wr(group):
    indicator_wr = WilliamsRIndicator(high=group['AdjHigh'], low=group['AdjLow'], close=group['AdjClose'])
    return indicator_wr.williams_r()

price_df['momentum_wr'] = price_df.groupby('stock_id').progress_apply(calculate_wr).reset_index(level=0, drop=True)

price_df

In [82]:
# price_df.to_feather("storage.feather")

In [ ]:
from ta.momentum import PercentageVolumeOscillator
tqdm.pandas()    
def calculate_pvo(group):
    indicator_pvo = PercentageVolumeOscillator(volume =group['DS_VOLUME'])
    group['momentum_pvo'] = indicator_pvo.pvo() 
    group['momentum_pvo_hist'] = indicator_pvo.pvo_hist() 
    group['momentum_pvo_signal'] = indicator_pvo.pvo_signal()
    return group

price_df = price_df.groupby('stock_id').progress_apply(calculate_pvo).reset_index(level=0, drop=True)

price_df

In [ ]:
from ta.momentum import PercentagePriceOscillator

def calculate_ppo(group):
    indicator_ppo = PercentagePriceOscillator(close=group['AdjClose'])
    return indicator_ppo.ppo()

price_df['momentum_ppo'] = price_df.groupby('stock_id').progress_apply(calculate_ppo).reset_index(level=0, drop=True)


def calculate_ppo_signal(group):
    indicator_ppo = PercentagePriceOscillator(close=group['AdjClose'])
    return indicator_ppo.ppo_signal()

price_df['momentum_ppo_signal'] = price_df.groupby('stock_id').progress_apply(calculate_ppo_signal).reset_index(level=0, drop=True)


def calculate_ppo_hist(group):
    indicator_ppo = PercentagePriceOscillator(close=group['AdjClose'])
    return indicator_ppo.ppo_hist()

price_df['momentum_ppo_hist'] = price_df.groupby('stock_id').progress_apply(calculate_ppo_hist).reset_index(level=0, drop=True)


price_df

In [ ]:
from ta.momentum import ROCIndicator

def calculate_roc(group):
    indicator_roc = ROCIndicator(close=group['AdjClose'])
    return indicator_roc.roc()

price_df['momentum_roc'] = price_df.groupby('stock_id').progress_apply(calculate_roc).reset_index(level=0, drop=True)

price_df

In [ ]:
from ta.others import CumulativeReturnIndicator

def calculate_cr(group):
    indicator_cr = CumulativeReturnIndicator(close=group['AdjClose'])
    return indicator_cr.cumulative_return()

price_df['others_cr'] = price_df.groupby('stock_id').progress_apply(calculate_cr).reset_index(level=0, drop=True)

price_df

In [ ]:
from ta.others import DailyLogReturnIndicator

def calculate_dlr(group):
    indicator_dlr = DailyLogReturnIndicator(close=group['AdjClose'])
    return indicator_dlr.daily_log_return()

price_df['others_dlr'] = price_df.groupby('stock_id').progress_apply(calculate_dlr).reset_index(level=0, drop=True)

price_df

In [ ]:
from ta.others import DailyReturnIndicator

def calculate_dr(group):
    indicator_dr = DailyReturnIndicator(close=group['AdjClose'])
    return indicator_dr.daily_return()

price_df['others_dr'] = price_df.groupby('stock_id').progress_apply(calculate_dr).reset_index(level=0, drop=True)

price_df

In [ ]:
# Sharpe_ratio

price_df['daily_return'] = price_df.groupby('stock_id')['AdjClose'].pct_change()


def calculate_sharpe_ratio(group):
    
    rolling = group['daily_return'].rolling(window = 252)

    mean_return = rolling.mean()
    volatility = rolling.std()
    sharpe_ratio = mean_return / volatility * np.sqrt(252)

    return sharpe_ratio

price_df['sharpe_ratio'] = price_df.groupby('stock_id').progress_apply(calculate_sharpe_ratio).reset_index(level=0, drop=True)

price_df


In [ ]:
# Skewness of daily returns

def calculate_skewness(group):
    
    rolling = group['daily_return'].rolling(window = 252)
    skewness = rolling.skew()
    
    return skewness

price_df['skewness'] = price_df.groupby('stock_id').progress_apply(calculate_skewness).reset_index(level=0, drop=True)

price_df

In [ ]:
# Kurtosis of daily returns

def calculate_kurtosis(group):
    
    rolling = group['daily_return'].rolling(window = 252)
    kurtosis = rolling.apply(lambda x: x.kurtosis(), raw=False)
    return kurtosis

price_df['kurtosis'] = price_df.groupby('stock_id').progress_apply(calculate_kurtosis).reset_index(level=0, drop=True)

price_df

In [ ]:
# Max_spread of daily returns

def calculate_max_spread(group):
    
    rolling = group['daily_return'].rolling(window = 252)
    max_return = rolling.max()
    min_return = rolling.min()
    max_spread = max_return-min_return

    return max_spread

price_df['max_spread'] = price_df.groupby('stock_id').progress_apply(calculate_max_spread).reset_index(level=0, drop=True)

price_df

In [ ]:
# Average daily returns

def calculate_avg_return(group):
    
    rolling = group['daily_return'].rolling(window = 252)
    avg_return = rolling.mean()

    return avg_return

price_df['avg_return'] = price_df.groupby('stock_id').progress_apply(calculate_avg_return).reset_index(level=0, drop=True)

price_df

In [ ]:
# daily volatility during last 50 trading days

def calculate_short_vola(group):
    
    rolling = group['daily_return'].rolling(window = 50)
    short_vola = rolling.std()

    return short_vola

price_df['short_vola'] = price_df.groupby('stock_id').progress_apply(calculate_short_vola).reset_index(level=0, drop=True)

price_df

In [ ]:
# daily volatility during last year

def calculate_long_vola(group):
    
    rolling = group['daily_return'].rolling(window = 252)
    long_vola = rolling.std()

    return long_vola

price_df['long_vola'] = price_df.groupby('stock_id').progress_apply(calculate_long_vola).reset_index(level=0, drop=True)

price_df

In [ ]:
# Difference between short- and long-term daily volatility

price_df['difference_vola'] = price_df['short_vola'] - price_df['long_vola']

price_df

In [ ]:
# trading days within last year
from datetime import timedelta

price_df['Date'] = pd.to_datetime(price_df['Date'])

def calculate_trade_days(group):

    group['year'] = group['Date'].dt.year
    year_counts = group['year'].value_counts()
    group['trade_days'] = group['year'].map(year_counts)
    return group

price_df = price_df.groupby('stock_id').progress_apply(calculate_trade_days).reset_index(level=0, drop=True)


price_df

In [24]:
price_df = price_df.drop(['AdjOpen','AdjHigh','AdjLow','AdjClose','DS_VOLUME'], axis=1)
price_df = price_df.drop(['year'], axis=1)
price_df.to_pickle('US_tech96.pkl.gz')

In [25]:
price_df.to_pickle('US_tech96.pkl.gz')

In [1]:
import pandas as pd

price_df = pd.read_feather("storage.feather")
price_df = price_df.drop(['AdjOpen','AdjHigh','AdjLow','AdjClose','DS_VOLUME'], axis=1)
# price_df = price_df.drop(['year'], axis=1)
price_df

,Date,stock_id,Avg50,Avg100,Avg200,volatility_atr,volatility_bbm,volatility_bbh,volatility_bbl,volatility_bbhi,...,momentum_kama,momentum_rsi,momentum_stoch_rsi,momentum_stoch_rsi_d,momentum_stoch_rsi_k,momentum_tsi,momentum_stoch,momentum_stoch_signal,momentum_uo,momentum_wr
0,2010-01-04,000001.SZ,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2010-01-05,000001.SZ,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2010-01-06,000001.SZ,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2010-01-07,000001.SZ,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2010-01-08,000001.SZ,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11766207,2024-09-13,920002.BJ,-0.146065,NaN,NaN,1.961467,43.6850,50.963564,36.406436,0.0,...,40.505793,34.732640,0.518042,0.057560,0.172681,-33.198454,24.538259,8.803819,32.944060,-75.461741
11766208,2024-09-18,920002.BJ,-0.237944,NaN,NaN,1.939219,43.1970,50.518615,35.875385,0.0,...,40.093329,30.937453,0.246222,0.142478,0.254755,-34.188140,4.353562,9.630607,30.319753,-95.646438
11766209,2024-09-19,920002.BJ,-0.231014,NaN,NaN,1.869989,42.5565,49.059651,36.053349,0.0,...,39.851915,32.393368,0.350498,0.266341,0.371587,-34.604136,8.379121,12.423647,30.955316,-91.620879
11766210,2024-09-20,920002.BJ,-0.249803,NaN,NaN,1.799276,41.8585,46.976614,36.740386,0.0,...,39.585996,30.962401,0.248009,0.302639,0.281576,-35.339663,1.362398,4.698360,30.808589,-98.637602


In [2]:
from tqdm import tqdm
import os

price_df['Date'] = pd.to_datetime(price_df['Date']) 
price_df.set_index(['Date', 'stock_id'], inplace=True) 
save_path = 'factors_by_name'

In [5]:


for column in tqdm(price_df.columns):
    # column_folder = os.path.join(save_path, column)
    os.makedirs(save_path, exist_ok=True)
    # for date, group in price_df.groupby(level=0): 
    single_df = price_df[[column]].reset_index('stock_id')
    file_name = os.path.join(save_path, f'{column}.feather')
    single_df.to_feather(file_name)

100%|██████████| 81/81 [02:03<00:00,  1.52s/it]
